# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [ ]:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value

print(sum)

8.382332347441762


#2. What is the 13th number yielded by the generator?



In [ ]:
generator = square_root_generator(13)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 3. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
%%capture
!pip install dlt[duckdb]

In [ ]:
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")
print(info)

info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
print(info)


Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707573522.462073 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707573523.0583904 is LOADED and contains no failed jobs


In [ ]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people").df()
display(peoples)

print("\n\n\n Sum of all peoples age:")
peoples["age"].sum()

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707573522.462073,EliMRz5tKcoDew,None
1,2,Person_2,27,City_A,1707573522.462073,Mne1Uq5+pFFm8Q,None
2,3,Person_3,28,City_A,1707573522.462073,oSS208tzV+PiGA,None
3,4,Person_4,29,City_A,1707573522.462073,aXpANK1f5fYnDQ,None
4,5,Person_5,30,City_A,1707573522.462073,tob2h/QeIDOZGQ,None
5,3,Person_3,33,City_B,1707573523.0583904,GJDZuesHQX2L+w,Job_3
6,4,Person_4,34,City_B,1707573523.0583904,ny/eOamLOv97TQ,Job_4
7,5,Person_5,35,City_B,1707573523.0583904,VM8caOZ5N4bMZw,Job_5
8,6,Person_6,36,City_B,1707573523.0583904,Mr+oV3LUyBcdKQ,Job_6
9,7,Person_7,37,City_B,1707573523.0583904,KGbKgAadcO5ZZA,Job_7





 Sum of all peoples age:


353

# 4. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [ ]:
peoples = conn.sql("TRUNCATE TABLE people")

In [ ]:

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="merge")
print(info)

info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="merge",
                    primary_key="id")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707573997.1942236 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707573997.9361145 is LOADED and contains no failed jobs


In [ ]:
print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people").df()
display(peoples)

print("\n\n\n Sum of all peoples age:")
peoples["age"].sum()




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707573997.1942236,+l3x3DU/HAEoAQ,None
1,2,Person_2,27,City_A,1707573997.1942236,uEfbEN5tSZLdqA,None
2,8,Person_8,38,City_B,1707573997.9361145,8E4mJNtCyS9hVw,Job_8
3,7,Person_7,37,City_B,1707573997.9361145,wfKUTSpeqlijOw,Job_7
4,5,Person_5,35,City_B,1707573997.9361145,tCJOEy8sm+2zkA,Job_5
5,4,Person_4,34,City_B,1707573997.9361145,FGH4POwJs+LUvw,Job_4
6,3,Person_3,33,City_B,1707573997.9361145,43zZD6XjukEqMQ,Job_3
7,6,Person_6,36,City_B,1707573997.9361145,L/hD04HCeC00zQ,Job_6





 Sum of all peoples age:


266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX